## forward, backword 하는 클래스를 만들어보자
- forward는 loss function 이라고 보면 되겠다.


In [3]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x*y
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

## 실제 사과와 오렌지 구매하는 부분

In [4]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
a_layer = MulLayer()
o_layer = MulLayer()
add_layer = AddLayer()
tax_layer = MulLayer()

# Forward
a_price = a_layer.forward(apple, apple_num)
o_price = o_layer.forward(orange, orange_num)
all_price = add_layer.forward(a_price, o_price)
price = tax_layer.forward(all_price, tax)

# Backward
dprice = 1
dall_price, dtax = tax_layer.backward(dprice)
da_price, do_price = add_layer.backward(dall_price)
d_orange, d_orange_num = o_layer.backward(do_price)
d_apple, d_apple_num = a_layer.backward(da_price)

print(price)
print(d_apple, d_apple_num, d_orange, d_orange_num, dtax)

715.0000000000001
2.2 110.00000000000001 3.3000000000000003 165.0 650


## ReLU와 Sigmoid 구현해보자

In [6]:
# ReLU

class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x<=0)
        out = x.copy()  # 값을 그대로 복사해주나 보다
        
        # x.copy에 대해 알아보기 위한 print문
        out[self.mask] = 0
        print('x: ', x)
        print('out: ', out)
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [7]:
x = np.array([[1., -.5], [-2., 3.]])
print(x)

mask = (x <= 0)
print(mask)
print(x[mask])

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]
[-0.5 -2. ]


In [8]:
# Sigmoid
class sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + exp(-x))
        self.out = out
        
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx
    

## 이번에는 affine, softmax 를 구현해보자


In [9]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        
        self.dW = None
        self.db = None
        
    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x,shape[0], -1)  # tensor를 단순한 matrix로 바꿔준다
        self.x = x
        
        out = np.dot(self.x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
# softmax + cross entropp error
class SoftmaxWithLoss: 
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx
    